In [2]:
!pip install transformers
!pip install datasets transformers huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.3 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split

In [4]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
data = ["I like you", "I hate this movie"]
sentiment_pipeline(data)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.9998695850372314},
 {'label': 'NEGATIVE', 'score': 0.9996687173843384}]

In [5]:
import torch
torch.cuda.is_available()

True

In [6]:
df = pd.read_csv('/content/file.csv')

In [7]:
df.head()

,Unnamed: 0,tweets,labels
0,0,ChatGPT: Optimizing Language Models for Dialog...,neutral
1,1,"Try talking with ChatGPT, our new AI system wh...",good
2,2,ChatGPT: Optimizing Language Models for Dialog...,neutral
3,3,"THRILLED to share that ChatGPT, our new model ...",good
4,4,"As of 2 minutes ago, @OpenAI released their ne...",bad


In [8]:
df.columns

Index(['Unnamed: 0', 'tweets', 'labels'], dtype='object')

In [9]:
df.drop(columns=['Unnamed: 0'], inplace=True)


In [10]:
df.columns

Index(['tweets', 'labels'], dtype='object')

In [11]:
df.shape

(219294, 2)

In [12]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

df['tweets'] = df['tweets'].apply(remove_html_tags)

df.head()

<ipython-input-12-8c0bf1227b18>:2: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a filename than HTML or XML.

If you meant to use Beautiful Soup to parse the contents of a file on disk, then something has gone wrong. You should open the file first, using code like this:

    filehandle = open(your filename)

You can then feed the open filehandle into Beautiful Soup instead of using the filename.

However, if you want to parse some data that happens to look like a filename, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  soup = BeautifulSoup(text, 'html.parser')
<ipython-input-12-8c0bf1227b18>:2: MarkupResemblesLocatorWarning: The input pas

,tweets,labels
0,ChatGPT: Optimizing Language Models for Dialog...,neutral
1,"Try talking with ChatGPT, our new AI system wh...",good
2,ChatGPT: Optimizing Language Models for Dialog...,neutral
3,"THRILLED to share that ChatGPT, our new model ...",good
4,"As of 2 minutes ago, @OpenAI released their ne...",bad


In [13]:
from tqdm import tqdm

tqdm.pandas()

# Define punctuation and the function to remove punctuation
punctuation = string.punctuation

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', punctuation))

# Apply remove_punctuation function to 'tweets' column with tqdm progress
df['tweets'] = df['tweets'].progress_apply(remove_punctuation)

# Lowercasing the text with tqdm progress
df['tweets'] = df['tweets'].progress_apply(lambda x: x.lower())

# Display the modified DataFrame
df.head()

100%|██████████| 219294/219294 [00:00<00:00, 440797.93it/s]


,tweets,labels
0,chatgpt optimizing language models for dialogu...,neutral
1,try talking with chatgpt our new ai system whi...,good
2,chatgpt optimizing language models for dialogu...,neutral
3,thrilled to share that chatgpt our new model o...,good
4,as of 2 minutes ago openai released their new ...,bad


In [14]:
df['label'] = df['labels'].map({'neutral': 0, 'good': 1, 'bad': 2})  # Convert sentiments to binary labels
df = df[['label', 'tweets']]

In [15]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)

In [16]:
dataset_split = dataset.train_test_split(test_size=0.3)

small_train_dataset = dataset_split['train'].shuffle(seed=42).select([i for i in range(20000)])

small_test_dataset = dataset_split['test'].shuffle(seed=42).select([i for i in range(5000)])

In [17]:
print("\nTraining dataset: \n")
print(small_train_dataset)

print("\nTraining dataset: \n")
print(small_test_dataset)


Training dataset: 

Dataset({
    features: ['label', 'tweets'],
    num_rows: 20000
})

Training dataset: 

Dataset({
    features: ['label', 'tweets'],
    num_rows: 5000
})


In [18]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [19]:
def preprocess_function(examples):
    return tokenizer(examples["tweets"], truncation=True, padding=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [20]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [21]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.4 MB/s eta 0:00:00


In [23]:
# Import necessary libraries
import evaluate

# Load accuracy and F1 metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

# Update compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')  # Use 'weighted' for multi-class F1
    return {"accuracy": accuracy, "f1": f1}

In [24]:
from huggingface_hub import notebook_login
notebook_login()

In [25]:
from transformers import TrainingArguments

repo_name = "chatgpt-model-on-sentiment-analysis"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="epoch",
    push_to_hub = True
)

In [27]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

<ipython-input-27-1284251935e0>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [28]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 1738827satvikadwaram (1738827satvikadwaram-iiitdm-kurnool) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.808500
1000,0.612500
1500,0.508500
2000,0.499800
2500,0.496300
3000,0.383000
3500,0.371500
4000,0.386300
4500,0.386500
5000,0.360500


TrainOutput(global_step=12500, training_loss=0.3192314636230469, metrics={'train_runtime': 1228.5124, 'train_samples_per_second': 81.399, 'train_steps_per_second': 10.175, 'total_flos': 2664428468516208.0, 'train_loss': 0.3192314636230469, 'epoch': 5.0})

In [29]:
trainer.evaluate()

Trainer is attempting to log a value of "{'f1': 0.8719038592372912}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.6402749419212341,
 'eval_accuracy': 0.871,
 'eval_f1': {'f1': 0.8719038592372912},
 'eval_runtime': 15.7642,
 'eval_samples_per_second': 317.175,
 'eval_steps_per_second': 39.647,
 'epoch': 5.0}

In [30]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1740597056.7e816ac340b4.227.0:   0%|          | 0.00/10.8k [00:00<?, ?B/s]

events.out.tfevents.1740598313.7e816ac340b4.227.1:   0%|          | 0.00/411 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Satvi/chatgpt-model-on-sentiment-analysis/commit/f5c7868f4dfc6cb9d49f49930617b1ab8ccf0418', commit_message='End of training', commit_description='', oid='f5c7868f4dfc6cb9d49f49930617b1ab8ccf0418', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Satvi/chatgpt-model-on-sentiment-analysis', endpoint='https://huggingface.co', repo_type='model', repo_id='Satvi/chatgpt-model-on-sentiment-analysis'), pr_revision=None, pr_num=None)

In [31]:
save_directory = "/content/sentiment_model_v1"

trainer.save_model(save_directory)


No files have been modified since last commit. Skipping to prevent empty commit.


In [35]:
from transformers import pipeline
sentiment_model = pipeline(model="Satvi/chatgpt-model-on-sentiment-analysis")
sentiment_model(["I like this move", "This movie is bad!"])

config.json:   0%|          | 0.00/769 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


[{'label': 'LABEL_0', 'score': 0.9948166012763977},
 {'label': 'LABEL_2', 'score': 0.9989370703697205}]